### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import os

### Import Data

In [4]:
# main folder
path = r'C:\Users\steve\Documents\Olist Marketplace Analysis'

# order products
prods_df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'), index_col = 0)

# product category translation
categories_df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'categories_checked.csv'), index_col = 0)

# order items
items_df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'order_items_checked.csv'), index_col = 0)

# orders
ords_df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'order_checked.csv'), index_col = 0)

# payments
pay_df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'order_payments_checked.csv'), index_col = 0)

# seller
seller_df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'sellers_checked.csv'), index_col = 0)

# customer
cust_df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'customers_checked.csv'), index_col = 0)

# reviews
reviews_df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'order_reviews_checked.csv'), index_col = 0)

### Analysis

The following file will merge all pertinent data. Merges were contducted in the following order and connecting via the following columns:
1. **Products** + **Translated Category Names** on *product_category_name*  
2. *Result of 1* + **Order Items** on *product_id*  
3. *Result of 2* + **Orders** on *order_id*  
4. *Result of 3* + **Order Payments** on *order_id*  
5. *Result of 4* + **Sellers** on *seller_id*  
6. *Result of 5* + **Customers** on *customer_id*  
7. *Result of 6* + **Reviews** on *order_id*  

#### MERGE: products_checked + category_checked

In [8]:
prods_df.head(2)

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0


In [9]:
categories_df.head(2)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories


In [10]:
# merge tables with left join, keeping all of order products
prods_translated_df = prods_df.merge(categories_df, on = 'product_category_name', how='left')

# moved joined translated col to be to the right of its spanish name col
col_to_move = prods_translated_df.pop('product_category_name_english')
prods_translated_df.insert(2, 'product_category_name_english', col_to_move)

# show merged df
prods_translated_df.head()

,product_id,product_category_name,product_category_name_english,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [11]:
# check value counts of new merged col
prods_translated_df['product_category_name_english'].value_counts(dropna = False)

product_category_name_english
bed_bath_table               3029
sports_leisure               2867
furniture_decor              2657
health_beauty                2444
housewares                   2335
                             ... 
tablets_printing_image          9
fashion_childrens_clothes       5
home_comfort_2                  5
security_and_services           2
cds_dvds_musicals               1
Name: count, Length: 72, dtype: int64

In [12]:
prods_translated_df.shape

(32951, 10)

#### MERGE: prods_translated + orders_items_checked

In [14]:
items_df.head(2)

,order_id,seller_id,product_id,shipping_limit_date,order_item_id,price,freight_value,total_price,total_freight,order_total
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,4244733e06e7ecb4970a6e2683c13e61,2017-09-19 09:45:35,1,58.9,13.29,58.9,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,e5f2d52b802189ee658865ca93d83a8f,2017-05-03 11:05:13,1,239.9,19.93,239.9,19.93,259.83


In [15]:
# merge tables
items_prods_df = items_df.merge(prods_translated_df, on = 'product_id', indicator = False)
items_prods_df.head()

,order_id,seller_id,product_id,shipping_limit_date,order_item_id,price,freight_value,total_price,total_freight,order_total,product_category_name,product_category_name_english,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,4244733e06e7ecb4970a6e2683c13e61,2017-09-19 09:45:35,1,58.90,13.29,58.90,13.29,72.19,cool_stuff,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,e5f2d52b802189ee658865ca93d83a8f,2017-05-03 11:05:13,1,239.90,19.93,239.90,19.93,259.83,pet_shop,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0
2,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,c777355d18b72b67abbeef9df44fd0fd,2018-01-18 14:48:30,1,199.00,17.87,199.00,17.87,216.87,moveis_decoracao,furniture_decor,59.0,695.0,2.0,3050.0,33.0,13.0,33.0
3,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,7634da152a4610f1595efa32f14722fc,2018-08-15 10:10:18,1,12.99,12.79,12.99,12.79,25.78,perfumaria,perfumery,42.0,480.0,1.0,200.0,16.0,10.0,15.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,ac6c3623068f30de03045865e4e10089,2017-02-13 13:57:51,1,199.90,18.14,199.90,18.14,218.04,ferramentas_jardim,garden_tools,59.0,409.0,1.0,3750.0,35.0,40.0,30.0


In [16]:
items_prods_df.shape

(112650, 19)

#### MERGE: items_prods_df + orders_checked

In [18]:
ords_df.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purchase_to_approved_hrs,approved_to_carrier_hrs,carrier_to_customer_hrs,purchase_to_delivered_hrs
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,0.18,56.80,145.50,202.48
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,30.71,11.11,288.95,330.77


In [19]:
# merge tables
ord_items_df = ords_df.merge(items_prods_df, on = 'order_id', indicator = True)
ord_items_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purchase_to_approved_hrs,approved_to_carrier_hrs,...,product_category_name,product_category_name_english,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,_merge
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,0.18,56.80,...,utilidades_domesticas,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0,both
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,30.71,11.11,...,perfumaria,perfumery,29.0,178.0,1.0,400.0,19.0,13.0,19.0,both
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,0.28,4.91,...,automotivo,auto,46.0,232.0,1.0,420.0,24.0,19.0,21.0,both
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,0.30,89.90,...,pet_shop,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,both
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.03,21.43,...,papelaria,stationery,38.0,316.0,4.0,250.0,51.0,15.0,15.0,both


In [20]:
ord_items_df['_merge'].value_counts(dropna = False)

_merge
both          112650
left_only          0
right_only         0
Name: count, dtype: int64

In [21]:
ord_items_df = ord_items_df.drop(columns = ['_merge'])

In [22]:
ord_items_df.shape

(112650, 30)

#### MERGE: ord_items_df + payment_checked

In [24]:
pay_df.head(2)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39


In [25]:
ord_pay_df = pay_df.merge(ord_items_df, on = 'order_id', indicator = True)
ord_pay_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,product_category_name,product_category_name_english,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,_merge
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,...,casa_construcao,home_construction,57.0,921.0,8.0,800.0,17.0,27.0,17.0,both
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,...,automotivo,auto,59.0,1274.0,2.0,150.0,16.0,6.0,11.0,both
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,...,perfumaria,perfumery,54.0,1536.0,2.0,250.0,20.0,8.0,21.0,both
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,...,cama_mesa_banho,bed_bath_table,60.0,188.0,1.0,1200.0,44.0,2.0,35.0,both
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45,15fd6fb8f8312dbb4674e4518d6fa3b3,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,...,cama_mesa_banho,bed_bath_table,54.0,120.0,1.0,835.0,40.0,9.0,33.0,both


In [26]:
ord_pay_df['_merge'].value_counts(dropna = False)

_merge
both          117601
left_only          0
right_only         0
Name: count, dtype: int64

In [27]:
ord_pay_df = ord_pay_df.drop(columns = ['_merge'])

In [28]:
ord_pay_df.shape

(117601, 34)

#### MERGE: ord_pay_df + sellers_checked

In [30]:
seller_df.head(2)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP


In [31]:
ord_sell_df = ord_pay_df.merge(seller_df, on = 'seller_id', how='left', indicator = True)
ord_sell_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,_merge
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,...,921.0,8.0,800.0,17.0,27.0,17.0,13321,salto,SP,both
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,...,1274.0,2.0,150.0,16.0,6.0,11.0,4660,sao paulo,SP,both
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,...,1536.0,2.0,250.0,20.0,8.0,21.0,9015,santo andre,SP,both
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,...,188.0,1.0,1200.0,44.0,2.0,35.0,13405,piracicaba,SP,both
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45,15fd6fb8f8312dbb4674e4518d6fa3b3,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,...,120.0,1.0,835.0,40.0,9.0,33.0,14940,ibitinga,SP,both


In [32]:
ord_sell_df['_merge'].value_counts(dropna = False)

_merge
both          117601
left_only          0
right_only         0
Name: count, dtype: int64

In [33]:
ord_sell_df = ord_sell_df.drop(columns = ['_merge'])

In [34]:
ord_sell_df.shape

(117601, 37)

#### MERGE: ord_sell_df + customers_checked

In [36]:
cust_df.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


In [37]:
ord_cust_df = ord_sell_df.merge(cust_df, on = 'customer_id', how='left', indicator = True)
ord_cust_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,_merge
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,...,27.0,17.0,13321,salto,SP,708ab75d2a007f0564aedd11139c7708,39801,teofilo otoni,MG,both
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,...,6.0,11.0,4660,sao paulo,SP,a8b9d3a27068454b1c98cc67d4e31e6f,2422,sao paulo,SP,both
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,...,8.0,21.0,9015,santo andre,SP,6f70c0b2f7552832ba46eb57b1c5651e,2652,sao paulo,SP,both
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,...,2.0,35.0,13405,piracicaba,SP,87695ed086ebd36f20404c82d20fca87,36060,juiz de fora,MG,both
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45,15fd6fb8f8312dbb4674e4518d6fa3b3,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,...,9.0,33.0,14940,ibitinga,SP,4291db0da71914754618cd789aebcd56,18570,conchas,SP,both


In [38]:
ord_cust_df['_merge'].value_counts(dropna = False)

_merge
both          117601
left_only          0
right_only         0
Name: count, dtype: int64

In [39]:
ord_cust_df = ord_cust_df.drop(columns = ['_merge'])

In [40]:
ord_cust_df.shape

(117601, 41)

#### MERGE: ord_cust_df + reviews_checked

In [42]:
reviews_df.head(2)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13


In [43]:
df = ord_cust_df.merge(reviews_df, on = 'order_id', how='left', indicator = True)
df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,_merge
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,...,39801,teofilo otoni,MG,4ce650e117a5b292bf5ea980a61eb61e,1.0,Não recomendo,"Produto com qualidade ruim, amassado e descasc...",2018-05-10 00:00:00,2018-05-15 23:38:00,both
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,...,2422,sao paulo,SP,ad2036caca61484fc435d7e673c020f2,5.0,NaN,NaN,2018-06-30 00:00:00,2018-07-03 11:23:16,both
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,...,2652,sao paulo,SP,bcddfa0f7d2e54cfc74bae4f01b84d92,5.0,NaN,Aprovada,2017-12-19 00:00:00,2017-12-22 17:55:30,both
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,...,36060,juiz de fora,MG,c37a0e51e5df82bdb55d38ff0ffb6af3,5.0,NaN,NaN,2017-12-21 00:00:00,2017-12-21 21:42:31,both
4,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,...,36060,juiz de fora,MG,5bc7680443b4ef765fec11a8201eeb0c,5.0,NaN,NaN,2017-12-21 00:00:00,2017-12-22 01:10:39,both


In [44]:
df['_merge'].value_counts(dropna = False)

_merge
both          117329
left_only        978
right_only         0
Name: count, dtype: int64

In [45]:
df = df.drop(columns = ['_merge'])

### Final ALL Merged Descriptive Analysis

In [47]:
df.shape

(118307, 47)

### Export Data

In [49]:
prods_translated_df.to_csv(os.path.join(path, '02 Data', 'Prepared Data','products_translations_merge.csv'))

In [50]:
df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'all_merged.pkl'))